In [1]:
import sqlite3
import pandas as pd

### Patent classification

In [2]:
conn = sqlite3.connect("/Volumes/jaekeun/uspto_2008_to_2018/done.db")
c = conn.cursor()

donn = sqlite3.connect("/Volumes/jaekeun/uspto_2008_to_2018/notdone.db")
d = donn.cursor()

In [3]:
with open("id.csv","r") as f:
    lines = f.readlines()
    bulkstring = "".join(lines)
    p_id_d = bulkstring.split(",")
    
with open("id_not.csv","r") as g:
    lines = g.readlines()
    bulkstring = "".join(lines)
    p_id_n = bulkstring.split(",")

### Retrieving patent_id from corp uuid

#### retrieving data wanted

In [4]:
query = c.execute("""SELECT * FROM (SELECT patent_id, GROUP_CONCAT(organization)
                                    FROM rawassignee 
                                    WHERE patent_id IN {} GROUP BY patent_id) AS sample
                   INNER JOIN uspc
                   ON uspc.patent_id = sample.patent_id""".format(tuple(p_id_d)))

print(c.fetchmany(50))

cols = [column[0] for column in query.description]
results_d= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

[('D606751', 'Apple Inc.', '1f1f4a4e-d755-11e8-86b9-1831bf2dfe44', 'D606751', 'D3', 'D3/269', 0), ('D606964', 'Apple Inc.', '23750828-d755-11e8-86b9-1831bf2dfe44', 'D606964', 'D14', 'D14/2037', 0), ('D606964', 'Apple Inc.', '23750829-d755-11e8-86b9-1831bf2dfe44', 'D606964', 'D14', 'D14/496', 1), ('D606964', 'Apple Inc.', '2375082a-d755-11e8-86b9-1831bf2dfe44', 'D606964', 'D14', 'D14/2033', 2), ('D606965', 'Apple Inc.', '23750929-d755-11e8-86b9-1831bf2dfe44', 'D606965', 'D14', 'D14/2037', 0), ('D606965', 'Apple Inc.', '2375092a-d755-11e8-86b9-1831bf2dfe44', 'D606965', 'D14', 'D14/496', 1), ('D606965', 'Apple Inc.', '2375092b-d755-11e8-86b9-1831bf2dfe44', 'D606965', 'D14', 'D14/2033', 2), ('D606966', 'Apple Inc.', '23bcda78-d755-11e8-86b9-1831bf2dfe44', 'D606966', 'D14', 'D14/2037', 0), ('D606966', 'Apple Inc.', '23bcda79-d755-11e8-86b9-1831bf2dfe44', 'D606966', 'D14', 'D14/496', 1), ('D606966', 'Apple Inc.', '23bcda7a-d755-11e8-86b9-1831bf2dfe44', 'D606966', 'D14', 'D14/2033', 2), ('D60

In [8]:
query = d.execute("""SELECT * FROM (SELECT patent_id, GROUP_CONCAT(organization)
                                    FROM rawassignee 
                                    WHERE patent_id IN {} GROUP BY patent_id) AS sample
                   INNER JOIN uspc
                   ON uspc.patent_id = sample.patent_id""".format(tuple(p_id_n)))

print(d.fetchmany(50))

cols = [column[0] for column in query.description]
results_n= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

[('D674709', 'Apple Inc.', 'c93c0b53-e0a6-11e8-86b9-1831bf2dfe44', 'D674709', 'D9', 'D9/737', 0), ('D674709', 'Apple Inc.', 'c93c0b54-e0a6-11e8-86b9-1831bf2dfe44', 'D674709', 'D9', 'D9/432', 1), ('D674813', 'Microsoft Corporation', 'cb7ed6e7-e0a6-11e8-86b9-1831bf2dfe44', 'D674813', 'D14', 'D14/487', 0), ('D674814', 'Microsoft Corporation', 'cb7ed7a7-e0a6-11e8-86b9-1831bf2dfe44', 'D674814', 'D14', 'D14/487', 0), ('8357008', 'CISCO Technology, Inc.', 'e9f99891-e0a6-11e8-86b9-1831bf2dfe44', '8357008', '439', '439/535', 0), ('8357008', 'CISCO Technology, Inc.', 'e9f99892-e0a6-11e8-86b9-1831bf2dfe44', '8357008', '174', '174/480', 1), ('8357008', 'CISCO Technology, Inc.', 'e9f99893-e0a6-11e8-86b9-1831bf2dfe44', '8357008', '174', '174/491', 2), ('8357317', 'Microsoft Corporation', '024cf9a7-e0a7-11e8-86b9-1831bf2dfe44', '8357317', '264', '264/129', 0), ('8357317', 'Microsoft Corporation', '024cf9a8-e0a7-11e8-86b9-1831bf2dfe44', '8357317', '264', '264/27', 1), ('8357317', 'Microsoft Corporatio

In [6]:
results_d.columns = ['patent_id', 'organization', 'uuid', 'patent_id2',
       'mainclass_id','subclass_id',
       'sequence']

results_d = results_d.drop(columns=["patent_id2"])
results_d.to_csv("done_patent_class.tsv", sep="\t")

In [15]:
results_n.columns = ['patent_id', 'organization', 'uuid', 'patent_id2',
       'mainclass_id','subclass_id',
       'sequence']

results_n = results_n.drop(columns=["patent_id2"])
results_n.to_csv("notdone_patent_class.tsv", sep="\t")

In [17]:
dataframes = [results_d, results_n]
results = pd.concat(dataframes, ignore_index=True)

465734